In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31962
2,Huelva,Confirmados PDIA 14 días,1502
3,Huelva,Tasa PDIA 14 días,"292,69053140284893"
4,Huelva,Confirmados PDIA 7 días,682
5,Huelva,Total Confirmados,32142
6,Huelva,Curados,29641
7,Huelva,Fallecidos,384


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31962.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8592.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 493 personas en los últimos 7 días 

Un positivo PCR cada 239 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31962.0,682.0,1502.0,513170.0,132.899429,292.690531,110.0
Huelva-Costa,18403.0,397.0,830.0,289548.0,137.110255,286.653681,70.0
Huelva (capital),8592.0,292.0,602.0,143837.0,203.007571,418.529307,46.0
Condado-Campiña,10380.0,218.0,526.0,156231.0,139.536968,336.680940,28.0
Sierra de Huelva-Andévalo Central,2829.0,64.0,141.0,67391.0,94.968171,209.226751,12.0
Valverde del Camino,609.0,34.0,72.0,12750.0,266.666667,564.705882,10.0
Punta Umbría,832.0,24.0,40.0,15355.0,156.300879,260.501465,5.0
Lepe,2140.0,13.0,27.0,27880.0,46.628407,96.843615,5.0
Aljaraque,1127.0,36.0,74.0,21474.0,167.644593,344.602775,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,179.0,7.0,14.0,2211.0,316.598824,633.197648,2.0
Valverde del Camino,609.0,34.0,72.0,12750.0,266.666667,564.705882,10.0
Niebla,239.0,5.0,22.0,4116.0,121.477162,534.499514,0.0
Palos de la Frontera,685.0,35.0,62.0,11742.0,298.075285,528.019077,4.0
San Juan del Puerto,524.0,24.0,45.0,9411.0,255.020720,478.163851,0.0
Huelva (capital),8592.0,292.0,602.0,143837.0,203.007571,418.529307,46.0
Almonte,1846.0,30.0,97.0,24507.0,122.414004,395.805280,3.0
Palma del Condado (La),1164.0,16.0,40.0,10801.0,148.134432,370.336080,3.0
Moguer,1313.0,30.0,79.0,21867.0,137.193031,361.274981,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Gibraleón,634.0,3.0,24.0,12737.0,23.553427,188.427416,1.0,0.125000
Villanueva de los Castillejos,163.0,1.0,5.0,2825.0,35.398230,176.991150,1.0,0.200000
San Bartolomé de la Torre,300.0,2.0,9.0,3761.0,53.177346,239.298059,0.0,0.222222
Niebla,239.0,5.0,22.0,4116.0,121.477162,534.499514,0.0,0.227273
Isla Cristina,2153.0,5.0,20.0,21393.0,23.372131,93.488524,3.0,0.250000
Nerva,178.0,1.0,4.0,5169.0,19.346102,77.384407,0.0,0.250000
Rociana del Condado,558.0,4.0,14.0,7939.0,50.384179,176.344628,0.0,0.285714
Aracena,531.0,4.0,13.0,8255.0,48.455482,157.480315,1.0,0.307692
Almonte,1846.0,30.0,97.0,24507.0,122.414004,395.805280,3.0,0.309278
